In [1]:
import numpy as np
import glob
import os

MODELNAME = "TestVAEwATT"

gameName = "LodeRunner"
rowLength = 32
numOfRows = 22

columnRefArray = np.array([np.arange(0, 32) for i in range(numOfRows+5)]).flatten()

## Generate Embedded LSTM Training Data

In [2]:
lrEmbeddingPath = f"Models/{MODELNAME}/LevelUnifiedRep/{gameName}"
lrEmbeddingPaths = sorted(glob.glob(f"{lrEmbeddingPath}/level*.npy"))

In [3]:
if os.path.isdir(f"Models/{MODELNAME}/LRLSTMData"):
    raise RuntimeError(f"Models/{MODELNAME}/LRLSTMData folder already exists. Make sure this is the right model. If it is delete this folder and re-run the code.")

os.mkdir(f"Models/{MODELNAME}/LRLSTMData")

padSize = rowLength * 3 # lode runner row length * 3 as paper uses previous 3 rows for lstm
N = padSize

sosArray = np.ones(shape=(1, 256)) * 9
eosArray = np.ones(shape=(1, 256)) * 5

xTrain = []
yTrain = []
xTrainTargetIn = []
columnRef = []

batchNum = 0

for i, levelEmbeddingPath in enumerate(lrEmbeddingPaths):

    levelEmbeddingArray = np.load(levelEmbeddingPath)

    for j in range(len(levelEmbeddingArray) - N):

        padLength = (N - j) if j < N else 0
        RowCutOff = 0 if j <= N else RowCutOff+1
        # print(f"RowCutOff: {RowCutOff}")
        # print(f"j: {j}, N: {N}")
        # print(j < N)
        # print(padLength)

        dataI = np.concatenate((np.zeros(shape=(padLength, 256)), levelEmbeddingArray[RowCutOff:j]), axis=0)
        # print(f"dataI shape: {dataI.shape}")

        dataT = levelEmbeddingArray[j:j+N]
        targetIn = np.concatenate((sosArray, dataT))
        targetOut = np.concatenate((dataT, eosArray))

        #levelIdx = np.concatenate((np.zeros(shape=(padLength)), columnRefArray[RowCutOff:j]), axis=0)
        levelIdx = np.array(columnRefArray[j:N+j])
        dataC = np.zeros(shape=(N, 256))
        for t in range(N): dataC[t][int(levelIdx[t])] = 1
        # print(f"dataC shape: {dataC.shape} : {dataC[-1]}")

        columnRef.append(dataC)
        xTrain.append(dataI)
        xTrainTargetIn.append(targetIn)
        yTrain.append(targetOut)

    if len(xTrain) / 32 >= 100.0:

        savePath = f"Models/{MODELNAME}/LRLSTMData/batch{batchNum}"

        np.save(f"{savePath}xTrain.npy", np.array(xTrain))
        np.save(f"{savePath}xTrainTargetIn.npy", np.array(xTrainTargetIn))
        np.save(f"{savePath}yTrain.npy", np.array(yTrain))
        np.save(f"{savePath}columnRef.npy", np.array(columnRef))

        xTrain = []
        yTrain = []
        xTrainTargetIn = []
        columnRef = []

        batchNum += 1

if len(xTrain) > 0:

    savePath = f"Models/{MODELNAME}/LRLSTMData/batch{batchNum}"

    np.save(f"{savePath}xTrain.npy", np.array(xTrain))
    np.save(f"{savePath}xTrainTargetIn.npy", np.array(xTrainTargetIn))
    np.save(f"{savePath}yTrain.npy", np.array(yTrain))
    np.save(f"{savePath}columnRef.npy", np.array(columnRef))

## Generate VGLC LSTM Training Data

In [9]:
lrStringLevelPaths = sorted(glob.glob("../data/vglc/Lode Runner/Processed/Level*.txt"))

In [11]:
if os.path.isdir(f"Models/{MODELNAME}/LRLSTMData"):
    raise RuntimeError(f"Models/{MODELNAME}/LRLSTMData folder already exists. Make sure this is the right model. If it is delete this folder and re-run the code.")

os.mkdir(f"Models/{MODELNAME}/LRLSTMData")

tiles = {
    "B" : 0,
    "b" : 1,
    "@" : 2,
    "." : 3,
    "-" : 4,
    "#" : 5,
    "G" : 6,
    "E" : 7,
    "M" : 8,
}

padSize = rowLength * 3 # lode runner row length * 3 as paper uses previous 3 rows for lstm
N = padSize

sosArray = np.ones(shape=(1)) * 9
eosArray = np.ones(shape=(1)) * 5

xTrain = []
yTrain = []
xTrainTargetIn = []
columnRef = []

batchNum = 0

for i, levelStringPath in enumerate(lrStringLevelPaths):

    with open(levelStringPath, "r") as f:
        levelString = np.array([list(line[:-1]) for line in f.readlines()]).flatten()
        levelString = np.array([tiles[tileChar] for tileChar in levelString])

    for j in range(len(levelString) - N):

        padLength = (N - j) if j < N else 0
        RowCutOff = 0 if j <= N else RowCutOff+1
        # print(f"RowCutOff: {RowCutOff}")
        # print(f"j: {j}, N: {N}")
        # print(j < N)
        # print(padLength)

        dataI = np.concatenate((np.zeros(shape=(padLength)), levelString[RowCutOff:j]), axis=0)
        # print(f"dataI shape: {dataI.shape}")

        dataT = levelString[j:j+N]
        targetIn = np.concatenate((sosArray, dataT))
        targetOut = np.concatenate((dataT, eosArray))

        #levelIdx = np.concatenate((np.zeros(shape=(padLength)), columnRefArray[RowCutOff:j]), axis=0)
        levelIdx = np.array(columnRefArray[j:N+j])
        dataC = np.zeros(shape=(N, 256))
        for t in range(N): dataC[t][int(levelIdx[t])] = 1
        # print(f"dataC shape: {dataC.shape} : {dataC[-1]}")

        columnRef.append(dataC)
        xTrain.append(dataI)
        xTrainTargetIn.append(targetIn)
        yTrain.append(targetOut)

    if len(xTrain) / 32 >= 100.0:

        savePath = f"Models/{MODELNAME}/LRLSTMData/batch{batchNum}"

        np.save(f"{savePath}xTrain.npy", np.array(xTrain))
        np.save(f"{savePath}xTrainTargetIn.npy", np.array(xTrainTargetIn))
        np.save(f"{savePath}yTrain.npy", np.array(yTrain))
        np.save(f"{savePath}columnRef.npy", np.array(columnRef))

        xTrain = []
        yTrain = []
        xTrainTargetIn = []
        columnRef = []

        batchNum += 1

if len(xTrain) > 0:

    savePath = f"Models/{MODELNAME}/LRLSTMData/batch{batchNum}"

    np.save(f"{savePath}xTrain.npy", np.array(xTrain))
    np.save(f"{savePath}xTrainTargetIn.npy", np.array(xTrainTargetIn))
    np.save(f"{savePath}yTrain.npy", np.array(yTrain))
    np.save(f"{savePath}columnRef.npy", np.array(columnRef))

RuntimeError: Models/VGCLLSTM/LRLSTMData folder already exists. Make sure this is the right model. If it is delete this folder and re-run the code.